Following: [text](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)
Compared to Recurrent Neural Networks (RNNs), the transformer model has proven to be superior in quality for many sequence-to-sequence tasks while being more parallelizable.